# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [1]:
import os
import requests
import re
import time
import threading
from bs4 import BeautifulSoup

In [2]:
class myThread (threading.Thread):
    def __init__(self, key_word_link, key_word, recursive):
        threading.Thread.__init__(self)
        self.key_word_link = key_word_link
        self.key_word = key_word
        self.recursive = recursive
    def run(self):
        WikiArticle(self.key_word_link, self.key_word, self.recursive, multithread=True)

def WikiArticle(key_word_link, key_word, recursive, multithread=False):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        '''
        
        Your code here
        
        '''
        dir_ = 'WikiArticle'
        if not os.path.exists(dir_):
            os.makedirs(dir_)
        
        with open('{}/{}.txt'.format(dir_, key_word.replace('/', '_')), 'w', encoding='utf8') as f_:
            for paragraph in content:
                f_.write(paragraph.get_text())
            

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        '''
        
        Your code here
        
        '''
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       # 外部連結的網址
                    a_keyword = link_string.get_text()  # 外部連結的中文名稱
                    external_link_dict[a_link] = a_keyword

                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            thread_list = []
            for k, v in external_link_dict.items():
                if multithread:
                    thread = myThread(k, v, recursive)
                    thread_list.append(thread)
                    thread.start()
                else:
                    WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                    
            if multithread:
                for thread in thread_list:
                    thread.join()

In [3]:
input_keyword = "蔡英文"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

蔡英文: %E8%94%A1%E8%8B%B1%E6%96%87
/wiki/%E8%94%A1%E8%8B%B1%E6%96%87


In [4]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 1

startTime = time.time()
WikiArticle(root_keyword_link, input_keyword, 0)
finishTime = time.time()
print(finishTime - startTime) # 正常情況的爬蟲所需時間

遞迴層[0] - /wiki/%E8%94%A1%E8%8B%B1%E6%96%87 (蔡英文)
遞迴層[1] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B ()
遞迴層[1] - /wiki/%E6%94%BF%E6%B2%BB%E4%BA%BA%E7%89%A9 (政治人物)
遞迴層[1] - /wiki/%E6%B0%91%E4%B8%BB%E9%80%B2%E6%AD%A5%E9%BB%A8 (民主進步黨)
遞迴層[1] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E5%9C%8B%E5%AE%B6%E5%85%83%E9%A6%96%E5%88%97%E8%A1%A8 (第14、15任)
遞迴層[1] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E7%B8%BD%E7%B5%B1 (中華民國總統)
遞迴層[1] - /wiki/%E6%B0%91%E4%B8%BB%E9%80%B2%E6%AD%A5%E9%BB%A8%E4%B8%BB%E5%B8%AD (民主進步黨主席)
遞迴層[1] - /wiki/%E8%87%BA%E5%8C%97%E5%B8%82 (臺北市)
遞迴層[1] - /wiki/%E4%B8%AD%E5%B1%B1%E5%8D%80_(%E8%87%BA%E5%8C%97%E5%B8%82) (中山區)
遞迴層[1] - /wiki/%E5%B1%8F%E6%9D%B1%E7%B8%A3 (屏東縣)
遞迴層[1] - /wiki/%E6%9E%8B%E5%B1%B1%E9%84%89 (枋山鄉)
遞迴層[1] - /wiki/%E5%AE%A2%E5%AE%B6%E8%A3%94%E8%87%BA%E7%81%A3%E4%BA%BA (客家裔臺灣人)
遞迴層[1] - /wiki/%E7%A6%8F%E4%BD%AC%E5%AE%A2 (福佬客)
遞迴層[1] - /wiki/%E8%87%BA%E7%81%A3%E5%8E%9F%E4%BD%8F%E6%B0%91%E6%97%8F (臺灣原住民族)
遞迴層[1] - /wiki/%E6%8E%92%E7%81%A3%E6%97%8F (排灣族)
遞迴

遞迴層[1] - /wiki/%E4%B8%81%E6%B8%9D%E6%B4%B2 (丁渝洲)
遞迴層[1] - /wiki/%E4%B8%AD%E5%9C%8B%E5%85%B1%E7%94%A2%E9%BB%A8 (中國共產黨)
遞迴層[1] - /wiki/%E4%B9%9D%E4%BA%8C%E5%85%B1%E8%AD%98 (九二共識)
遞迴層[1] - /wiki/%E4%B8%AD%E5%9C%8B%E5%A4%A7%E9%99%B8 (中國大陸)
遞迴層[1] - /wiki/%E5%B0%8F%E4%B8%89%E9%80%9A (小三通)
遞迴層[1] - /wiki/%E4%B8%80%E9%82%8A%E4%B8%80%E5%9C%8B (一邊一國)
遞迴層[1] - /wiki/%E8%87%BA%E7%81%A3%E5%9C%B0%E5%8D%80%E8%88%87%E5%A4%A7%E9%99%B8%E5%9C%B0%E5%8D%80%E4%BA%BA%E6%B0%91%E9%97%9C%E4%BF%82%E6%A2%9D%E4%BE%8B (臺灣地區與大陸地區人民關係條例)
遞迴層[1] - /wiki/%E5%85%A9%E5%B2%B8%E5%AE%9A%E6%9C%9F%E8%88%AA%E7%B7%9A (兩岸春節包機直航)
遞迴層[1] - /wiki/%E8%BC%BF%E8%AB%96%E8%AA%BF%E6%9F%A5 (輿論調查)
遞迴層[1] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E7%B8%BD%E7%B5%B1%E5%BA%9C%E5%9C%8B%E7%AD%96%E9%A1%A7%E5%95%8F (中華民國總統府國策顧問)
遞迴層[1] - /wiki/2004%E5%B9%B4%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E7%B8%BD%E7%B5%B1%E9%81%B8%E8%88%89 (還能執政)
遞迴層[1] - /wiki/%E5%9C%8B%E5%8B%99%E9%99%A2%E8%87%BA%E7%81%A3%E4%BA%8B%E5%8B%99%E8%BE%A6%E5%85%AC%E5%AE%A4 (國務院

遞迴層[1] - /wiki/%E7%99%BD%E5%AE%AE (白宮)
遞迴層[1] - /wiki/%E7%BE%8E%E5%9C%8B%E5%9C%8B%E5%AE%B6%E5%AE%89%E5%85%A8%E6%9C%83%E8%AD%B0 (美國國家安全會議)
遞迴層[1] - /wiki/%E7%BE%8E%E5%9C%8B%E5%9C%8B%E5%8B%99%E9%99%A2 (美國國務院)
遞迴層[1] - /wiki/%E7%BE%8E%E5%9C%8B%E5%89%AF%E5%9C%8B%E5%8B%99%E5%8D%BF (美國副國務卿)
遞迴層[1] - /wiki/%E7%B4%90%E7%B4%84 (紐約)
遞迴層[1] - /wiki/%E8%87%BA%E7%81%A3%E8%A3%94%E7%BE%8E%E5%9C%8B%E4%BA%BA (臺灣裔美國人)
遞迴層[1] - /wiki/%E6%97%A5%E6%9C%AC%E5%85%A7%E9%96%A3%E7%B8%BD%E7%90%86%E5%A4%A7%E8%87%A3 (日本內閣總理大臣)
遞迴層[1] - /wiki/%E5%AE%89%E5%80%8D%E6%99%89%E4%B8%89 (安倍晉三)
遞迴層[1] - /wiki/%E5%9C%8B%E6%B0%91%E9%BB%A8%E4%B8%AD%E5%A4%AE (國民黨中央)
遞迴層[1] - /wiki/%E4%B8%AD%E5%9C%8B%E5%9C%8B%E6%B0%91%E9%BB%A8%E5%85%A8%E5%9C%8B%E4%BB%A3%E8%A1%A8%E5%A4%A7%E6%9C%83 (臨時全國代表大會)
遞迴層[1] - /wiki/%E6%8F%9B%E6%9F%B1%E7%88%AD%E8%AD%B0 (廢止洪秀柱總統選舉提名資格)
遞迴層[1] - /wiki/%E6%96%B0%E5%8C%97%E5%B8%82%E9%95%B7 (新北市長)
遞迴層[1] - /wiki/2010%E5%B9%B4%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E7%9B%B4%E8%BD%84%E5%B8%82%E5%B8%82%E9%95%B7%E6%9A%

遞迴層[1] - /wiki/%E5%A4%96%E4%BA%A4%E6%94%BF%E7%AD%96_(%E9%9B%9C%E8%AA%8C) (外交政策)
遞迴層[1] - /wiki/FT_100%E5%85%A8%E7%90%83%E7%9F%A5%E8%AD%98%E5%88%86%E5%AD%90 (FT 100全球知識分子)
遞迴層[1] - /wiki/%E5%AF%8C%E6%AF%94%E5%A3%AB (富比士)
遞迴層[1] - /wiki/%E4%B8%96%E7%95%8C%E7%99%BE%E5%90%8D%E6%AC%8A%E5%A8%81%E5%A5%B3%E6%80%A7 (世界百名權威女性)
遞迴層[1] - /wiki/%E8%B2%A1%E6%98%9F_(%E9%9B%9C%E8%AA%8C) (財星)
遞迴層[1] - /wiki/%E6%B3%9B%E8%97%8D (泛藍)
遞迴層[1] - /wiki/%E8%95%B9%E8%8F%9C (空心菜)
遞迴層[1] - /wiki/%E8%A7%A3%E6%94%BE%E5%8F%B0%E6%B9%BE (武統臺灣)
遞迴層[1] - /wiki/%E8%89%A6%E9%9A%8ACollection (艦隊Collection)
遞迴層[1] - /wiki/%E9%9C%A7%E5%B3%B6%E8%99%9F%E6%88%B0%E8%89%A6 (霧島)
遞迴層[1] - /wiki/%E9%BB%8F%E5%9C%9F%E4%BA%BA (黏土人)
遞迴層[1] - /wiki/%E5%9C%8B%E6%B0%91%E6%94%BF%E5%BA%9C%E4%B8%BB%E5%B8%AD (國民政府委員會主席)
遞迴層[1] - /wiki/Template:%E4%B8%89%E7%9A%87%E4%BA%94%E5%B8%9D (三皇五帝)
遞迴層[1] - /wiki/Template:%E5%A4%8F%E6%9C%9D%E5%90%9B%E4%B8%BB (夏)
遞迴層[1] - /wiki/Template:%E5%95%86%E6%9C%9D%E5%90%9B%E4%B8%BB (商)
遞迴層[1] - /wiki/Template:%E5%9

In [5]:
startTime = time.time()
WikiArticle(root_keyword_link, input_keyword, 0, multithread=True)
finishTime = time.time()
print(finishTime - startTime) # 利用 thread 的爬蟲所需時間

遞迴層[0] - /wiki/%E8%94%A1%E8%8B%B1%E6%96%87 (蔡英文)
遞迴層[1] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B ()遞迴層[1] - /wiki/%E6%94%BF%E6%B2%BB%E4%BA%BA%E7%89%A9 (政治人物)

遞迴層[1] - /wiki/%E6%B0%91%E4%B8%BB%E9%80%B2%E6%AD%A5%E9%BB%A8 (民主進步黨)
遞迴層[1] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E5%9C%8B%E5%AE%B6%E5%85%83%E9%A6%96%E5%88%97%E8%A1%A8 (第14、15任)
遞迴層[1] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E7%B8%BD%E7%B5%B1 (中華民國總統)
遞迴層[1] - /wiki/%E6%B0%91%E4%B8%BB%E9%80%B2%E6%AD%A5%E9%BB%A8%E4%B8%BB%E5%B8%AD (民主進步黨主席)
遞迴層[1] - /wiki/%E8%87%BA%E5%8C%97%E5%B8%82 (臺北市)
遞迴層[1] - /wiki/%E4%B8%AD%E5%B1%B1%E5%8D%80_(%E8%87%BA%E5%8C%97%E5%B8%82) (中山區)
遞迴層[1] - /wiki/%E5%B1%8F%E6%9D%B1%E7%B8%A3 (屏東縣)
遞迴層[1] - /wiki/%E6%9E%8B%E5%B1%B1%E9%84%89 (枋山鄉)遞迴層[1] - /wiki/%E5%AE%A2%E5%AE%B6%E8%A3%94%E8%87%BA%E7%81%A3%E4%BA%BA (客家裔臺灣人)

遞迴層[1] - /wiki/%E7%A6%8F%E4%BD%AC%E5%AE%A2 (福佬客)
遞迴層[1] - /wiki/%E8%87%BA%E7%81%A3%E5%8E%9F%E4%BD%8F%E6%B0%91%E6%97%8F (臺灣原住民族)遞迴層[1] - /wiki/%E6%8E%92%E7%81%A3%E6%97%8F (排灣族)

遞迴

遞迴層[1] - /wiki/%E4%B8%AD%E5%9C%8B%E5%85%B1%E7%94%A2%E9%BB%A8 (中國共產黨)
遞迴層[1] - /wiki/%E4%B9%9D%E4%BA%8C%E5%85%B1%E8%AD%98 (九二共識)遞迴層[1] - /wiki/%E4%B8%AD%E5%9C%8B%E5%A4%A7%E9%99%B8 (中國大陸)
遞迴層[1] - /wiki/%E5%B0%8F%E4%B8%89%E9%80%9A (小三通)

遞迴層[1] - /wiki/%E4%B8%80%E9%82%8A%E4%B8%80%E5%9C%8B (一邊一國)
遞迴層[1] - /wiki/%E8%87%BA%E7%81%A3%E5%9C%B0%E5%8D%80%E8%88%87%E5%A4%A7%E9%99%B8%E5%9C%B0%E5%8D%80%E4%BA%BA%E6%B0%91%E9%97%9C%E4%BF%82%E6%A2%9D%E4%BE%8B (臺灣地區與大陸地區人民關係條例)
遞迴層[1] - /wiki/%E5%85%A9%E5%B2%B8%E5%AE%9A%E6%9C%9F%E8%88%AA%E7%B7%9A (兩岸春節包機直航)
遞迴層[1] - /wiki/%E8%BC%BF%E8%AB%96%E8%AA%BF%E6%9F%A5 (輿論調查)
遞迴層[1] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E7%B8%BD%E7%B5%B1%E5%BA%9C%E5%9C%8B%E7%AD%96%E9%A1%A7%E5%95%8F (中華民國總統府國策顧問)
遞迴層[1] - /wiki/2004%E5%B9%B4%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E7%B8%BD%E7%B5%B1%E9%81%B8%E8%88%89 (還能執政)
遞迴層[1] - /wiki/%E5%9C%8B%E5%8B%99%E9%99%A2%E8%87%BA%E7%81%A3%E4%BA%8B%E5%8B%99%E8%BE%A6%E5%85%AC%E5%AE%A4 (國務院臺灣事務辦公室)
遞迴層[1] - /wiki/2004%E5%B9%B4%E4%B8%AD%E8

遞迴層[1] - /wiki/%E5%8F%8C%E5%85%B3 (雙關)遞迴層[1] - /wiki/%E4%BA%8C%E6%AC%A1%E5%A4%A7%E6%88%B0 (二次大戰)

遞迴層[1] - /wiki/%E4%B8%8D%E5%88%97%E9%A1%9B%E6%88%B0%E5%BD%B9 (英倫空戰)
遞迴層[1] - /wiki/%E9%99%B3%E5%BB%BA%E4%BB%81 (陳建仁)
遞迴層[1] - /wiki/%E7%8E%8B%E5%A6%82%E7%8E%84 (王如玄)遞迴層[1] - /wiki/%E6%B0%91%E5%9C%8B%E9%BB%A8 (民國黨)

遞迴層[1] - /wiki/%E5%BE%90%E6%AC%A3%E7%91%A9 (徐欣瑩)遞迴層[1] - /wiki/%E8%A1%8C%E6%94%BF%E9%99%A2 (行政院)
遞迴層[1] - /wiki/%E6%9E%97%E5%85%A8%E5%85%A7%E9%96%A3 (內閣)
遞迴層[1] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E5%9C%8B%E6%97%97 (中華民國國旗)

遞迴層[1] - /wiki/%E5%AD%AB%E4%B8%AD%E5%B1%B1 (國父)
遞迴層[1] - /wiki/%E7%AB%8B%E6%B3%95%E9%99%A2%E9%99%A2%E9%95%B7 (立法院院長)
遞迴層[1] - /wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E5%9C%8B%E7%92%BD (中華民國國璽)遞迴層[1] - /wiki/%E7%BE%8E%E5%9C%8B%E8%81%AF%E9%82%A6%E6%94%BF%E5%BA%9C (美國聯邦政府)
遞迴層[1] - /wiki/%E5%8E%9F%E4%BD%8F%E6%B0%91%E6%97%8F%E6%97%A5 (原住民族日)

遞迴層[1] - /wiki/%E8%87%BA%E7%81%A3%E6%97%A5%E6%B2%BB%E6%99%82%E6%9C%9F (日治時期)遞迴層[1] - /wiki/%E7%9A%87%E6%B0%91

遞迴層[1] - /wiki/Template:%E6%99%89%E6%9C%9D%E5%90%9B%E4%B8%BB (晉)
遞迴層[1] - /wiki/Template:%E4%BA%94%E8%83%A1%E5%8D%81%E5%85%AD%E5%9C%8B%E5%90%9B%E4%B8%BB (十六國)
遞迴層[1] - /wiki/Template:%E5%8D%97%E6%9C%9D%E5%90%9B%E4%B8%BB (南朝)遞迴層[1] - /wiki/Template:%E5%85%83%E9%AD%8F%E5%90%9B%E4%B8%BB (元魏)
遞迴層[1] - /wiki/Template:%E5%8C%97%E9%BD%90%E5%90%9B%E4%B8%BB (北齊)

遞迴層[1] - /wiki/Template:%E5%8C%97%E5%91%A8%E5%90%9B%E4%B8%BB (北周)遞迴層[1] - /wiki/Template:%E9%9A%8B%E6%9C%9D%E5%90%9B%E4%B8%BB (隋)
遞迴層[1] - /wiki/Template:%E5%94%90%E6%9C%9D%E5%90%9B%E4%B8%BB (唐)

遞迴層[1] - /wiki/Template:%E4%BA%94%E4%BB%A3%E5%90%9B%E4%B8%BB (五代)
遞迴層[1] - /wiki/Template:%E5%8D%81%E5%9C%8B%E5%90%9B%E4%B8%BB (十國)
遞迴層[1] - /wiki/Template:%E5%AE%8B%E6%9C%9D%E5%90%9B%E4%B8%BB (宋)
遞迴層[1] - /wiki/Template:%E8%A5%BF%E5%A4%8F%E5%90%9B%E4%B8%BB (西夏)
遞迴層[1] - /wiki/Template:%E8%BE%BD%E6%9C%9D%E5%90%9B%E4%B8%BB (遼)
遞迴層[1] - /wiki/Template:%E9%87%91%E6%9C%9D%E5%90%9B%E4%B8%BB (金)
遞迴層[1] - /wiki/Template:%E5%85%83%E6%9C%9D%E5%90%9B%E4